# Airbnb a Berlino
### *Data processing e exploratory data analytics sulle attività di Airbnb a Berlino.*
Airbnb è un portale online che mette in contatto persone in cerca di un alloggio o di una camera per brevi periodi,
con persone che dispongono di uno spazio extra da affittare, generalmente privati.

Ad oggi Airbnb offre alloggi in oltre **26.000 città** in **192 paesi** con più di **10 milioni di notti prenotate** in tutto il mondo. <br>
Gli annunci includono sistemazioni quali stanze private, interi appartamenti, castelli e ville,
ma anche barche, baite, case sugli alberi, igloo, isole private e qualsiasi altro tipo di alloggio